In [34]:
import numpy as np
import openpyxl

class Client:
    def __init__(self, client_id, arrival_time, miu, theta, st_is_fixed_exp):
        self.client_id = client_id
        self.arrival_time = arrival_time
        self.service_time = np.random.exponential(1 / miu)
        self.drop_time = theta if st_is_fixed_exp else np.random.exponential(theta)


class Event:
    def __init__(self, client_id, event_time, status=0):
        self.client_id = client_id
        self.status = status
        self.event_time = event_time


class Server:
    def __init__(self, queue_size):
        self.queue_last_change = 0.0
        self.avg_num_client_q = 0.0
        self.num_exited = 0

        # Doesn't implement this feature
        # self.num_blocked = 0
        # self.num_dropped = 0

        # 0: Busy
        # 1: Idle
        # 2: Sleep
        self.state = 2
        self.max_events = 5

        self.queue_size = queue_size
        self.clients_queue = []
        self.events = []
        self.all_clients = []

    # Just for adding new event in server.events and sort the event first
    def add_events(self, new_client, event_time, event_status):
        new_event = Event(new_client.client_id, event_time, event_status)
        # Keep events array sorted in order to event_time  (Binary Search)
        i = 0
        j = len(self.events)
        while i < j:
            mid = (i + j) // 2
            if self.events[mid].event_time < new_event.event_time:
                i = mid + 1
            else:
                j = mid
        self.events.insert(i, new_event)

    # this is only for adding new client to queue
    def handle_new_client(self, new_client, time):
        self.all_clients.append(new_client)

        if len(self.clients_queue) == self.queue_size:
            self.num_blocked += 1
        else:
            event_time = 0
            event_status = 0
            
            # We didn't implement dropping in queue
            # so customer will always be served by system

            # if len(self.clients_queue) == 0:
            #     event_time = new_client.arrival_time + new_client.service_time
            #     event_status = 2
            # else:
            #     # first add all client as dropping to queue,but
            #     # after exit head of queue next client will change
            #     # status (You can check handle_events function)
            #     event_time = new_client.arrival_time + new_client.drop_time
            #     event_status = 1

            event_time = new_client.arrival_time + new_client.service_time
            self.avg_num_client_q += abs((time - self.queue_last_change) * (len(self.clients_queue)))
            self.queue_last_change = time

            self.clients_queue.append(new_client)
            self.add_events(new_client, event_time, event_status)

    def handle_events(self, time):
        event = self.events.pop()
        client = self.clients_queue.pop()

        print([x.client_id for x in self.events])
        print([x.client_id for x in self.clients_queue])
        print(client.client_id)
        print('Handling current user:', client.client_id)

        self.num_exited += 1
        self.avg_num_client_q += abs((event.event_time - self.queue_last_change) * (len(self.clients_queue)))
        self.queue_last_change = event.event_time
        self.clients_queue = [x for x in self.clients_queue if x.client_id != client.client_id]
        self.events = [x for x in self.events if x.client_id != client.client_id]



def simulator(num_customers, lambda_, miu, theta, queue_size, sleep_time, st_is_fixed_exp):
    time = 0
    server = Server(queue_size)
    client_id = 0

    # TODO: how to simulate arriving task and serving task simultaneously
    while client_id < num_customers:
        # Server is sleep
        # Waiting the queue to be filled
        if server.state == 2 and len(server.clients_queue) < server.max_events:
            server.handle_new_client(Client(client_id, time, miu, theta, st_is_fixed_exp), time)
            client_id += 1
            time += np.random.exponential(1 / lambda_)
            print('Adding new client to the sleep queue, ', 'Client id:', client_id)
            print('Server queue: ', [x.client_id for x in server.clients_queue])

        # Server's queue is filled
        # Change state from sleep to busy
        elif server.state == 2 and len(server.clients_queue) >= server.max_events:
            server.state = 0
            time += np.random.exponential(1 / lambda_)
            server.handle_events(time)
        
        # Changing to server idle 
        elif server.state == 0 and time + sleep_time < time + np.random.exponential(1 / lambda_):
            server.state = 1
        
        # Server state is busy
        elif server.state == 0:
            server.handle_new_client(Client(client_id, time, miu, theta, st_is_fixed_exp), time)
            client_id += 1
            time += np.random.exponential(1 / lambda_)
            server.handle_events(time)
            
    while len(server.clients_queue) != 0:
        server.handle_events(time)

    # print()
    print('Simulation:  ','Nc:', server.avg_num_client_q / time)
    return server.avg_num_client_q / time


In [35]:
if __name__ == "__main__":
    miu = 1  # service rate
    theta = 3  # deadline rate or deadline time
    lambda_ = 0.1  # arrival rate of customers
    queue_size = 14  # size of queue
    st_is_fixed_exp = False  # or False
    num_customers = 10  # number of customers in simulatior
    sleep_time = np.random.exponential(1 / lambda_)

    for st_is_fixed_exp in [True]:
        lambda_ = 0.1
        sim_nc= simulator(num_customers, lambda_, miu, theta, queue_size, sleep_time, st_is_fixed_exp)

        lambda_ += 0.1

Adding new client to the sleep queue,  Client id: 1
Server queue:  [0]
Adding new client to the sleep queue,  Client id: 2
Server queue:  [0, 1]
Adding new client to the sleep queue,  Client id: 3
Server queue:  [0, 1, 2]
Adding new client to the sleep queue,  Client id: 4
Server queue:  [0, 1, 2, 3]
Adding new client to the sleep queue,  Client id: 5
Server queue:  [0, 1, 2, 3, 4]
[0, 1, 2, 3]
[0, 1, 2, 3]
4
Handling current user: 4
[0, 1, 2, 3]
[0, 1, 2, 3]
5
Handling current user: 5
[0, 1, 2, 3]
[0, 1, 2, 3]
6
Handling current user: 6
[0, 1, 2, 3]
[0, 1, 2, 3]
7
Handling current user: 7
[0, 1, 2, 3]
[0, 1, 2, 3]
8
Handling current user: 8
[0, 1, 2, 3]
[0, 1, 2, 3]
9
Handling current user: 9
[0, 1, 2]
[0, 1, 2]
3
Handling current user: 3
[0, 1]
[0, 1]
2
Handling current user: 2
[0]
[0]
1
Handling current user: 1
[]
[]
0
Handling current user: 0
Simulation:   Nc: 6.170541531212742
